### Downloading data

For downloading frames and their corresponding annotations. You'll need to provide your TATOR API key, which you can get 
from the TATOR website (see docs), to be able to download data associated with this project (70). Set the ROOT directory 
so all data will be saved in a specific location:

```python
ROOT
   |
   JPEGImages
         |
         Image_1.jpg
         Image_2.jpg
   |
   Annotations
         |
         Label_2.xml
         Label_2.xml
```

Ensure that you're token is not in the notebook when making a commit!

### Installing

If you don't have the packages, run this cell and then restart the kernel

In [ ]:
!pip install tator tqdm pandas numpy pascal-voc-writer

### Imports

Install these using pip or conda

In [ ]:
import os
import json
import glob
import shutil
from tqdm import tqdm

import tator
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

from concurrent.futures import ThreadPoolExecutor
from pascal_voc_writer import Writer

In [ ]:
def download_image(frame, media_name, media_dir, api, media_id, download=False):
    """
    Function is used to download specific frames into
    the corresponding media folder.
    """
    # Location of file on local machine
    dst = media_dir + media_name.split(".")[0] + "_" + str(frame) + ".jpg"
    
    # If it doesn't already exist, download, move.
    if not os.path.exists(dst) and download:
        src = api.get_frame(media_id, frames=[frame])
        shutil.move(src, dst)
        
    return dst

### Setting Up

Here we're setting the root directory where data will be downloaded. By default, it's just a folder called Data, located in the root of the repository. Then we create sub-folders containing Images, and Annotations. The basename for each image and annotation will correspond to each other, and just differ based on the file extension.

Then we get the TATOR API token, which should be stored as an environmental variable.

In [ ]:
# Root data location 
ROOT = "./Data/"
IMAGE_DIR = ROOT + "JPEGImages/"
LABEL_DIR = ROOT + "Annotations/"

# This will create folders
os.makedirs(ROOT, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs(LABEL_DIR, exist_ok=True)

# Grabbing token from environmental variable
MY_TOKEN = os.getenv("TATOR_TOKEN")

# Setting the api given the token, sanity check
api = tator.get_api(host='https://cloud.tator.io', token=MY_TOKEN)

### Download?

If you don't want to download the images, set this variable to False

In [ ]:
# Boolean flag for downloading images
DOWNLOAD = False

### Project ID

The project ID can be found in the URL of the project on TATOR. You can only access specific projects given your API token.

In [ ]:
PROJECT_ID = 70

### Media of Interest

Here we're specifying the media we're interested in. Currently you have to add the name of the media by hand to create a list. However, you can also do some scripting to get all the media using `get_media_list`, and subset it somehow.

In [ ]:
# A list of the media we're intrested in
MEDIA_LIST = [
                "Some_Media_Name.mov"
             ]

In [ ]:
# Grabbing all media from the project
medias = api.get_media_list(PROJECT_ID)
medias_names = [m.name for m in medias] 

print(f"Number of medias: {len(medias)}")

In [ ]:
for media in MEDIA_LIST:
    if media in medias_names:
        print(f"This media WAS found: {media}")
    else:
        print(f"This media WASN'T found: {media}")

### Getting the Localizations for the Media of Interest

Previously, we specified the media we're interested in, in `MEDIA_LIST`. Now we're getting all of the localizations that are associated with each of those medias. Note that there is difference in how TATOR stores localization annotations made using the Track Tool, and using the Create Localization Tool.

In [ ]:
# A list of all class categories for all media
ALL_CLASSES_LIST = []

In [ ]:
# List to hold all media localization information
media_w_localizations = []

# Looping through each media, collecting information
# from the media that we're intrested in
for media in tqdm(medias):
    
    if media.name not in MEDIA_LIST:
        continue
        
    print("Collecting metadata for: ", media.name)
    
    # All localizations and tracks for this media 
    localizations = api.get_localization_list(PROJECT_ID, media_id=[media.id])
    tracks = api.get_state_list(PROJECT_ID, media_id=[media.id])
    
    # Tracks can contain localization information, but not always
    # as some tracks are for other things. Here we look for 
    # the class categories stored in eiter track localizations
    # (if they exist), and in the actual localizations
    
    # A dict containing the class category for this
    # media's localization, based on it's ID
    loc_id_to_class = {}
    
    # A list to hold all localizations for this media,
    # whether they are from tracks or otherwise
    media_localizations = []
    
    # Loop trhough the tracks
    for track in tracks:
        # Loop through all track localizations
        for localization in track.localizations:
            if "ScientifcName" in localization.attributes:
                class_category = localization.attributes['ScientificName']
                loc_id_to_class[localization.id] = class_category
                media_localizations.append(localization)
                ALL_CLASSES_LIST.append(class_category)
    
    # Now we look at all the actual localizations that are not tracks
    for localization in localizations:
        if "ScientificName" in localization.attributes:
            class_category = localization.attributes['ScientificName']
            loc_id_to_class[localization.id] = class_category
            media_localizations.append(localization)
            ALL_CLASSES_LIST.append(class_category)
            
    # Storing the media metadata we'll need later and the localizations
    media_w_localizations.append([media.name.replace(":", "_"),
                                  media.id,
                                  media.width,
                                  media.height,
                                  media_localizations,
                                  loc_id_to_class])

In [ ]:
# Show the distribution of class categories in these media
pd.DataFrame(ALL_CLASSES_LIST).value_counts().plot(kind='bar')

### Checking Localizations

If you notice, we store the localizations in the 4th index. So in this cell, we're accessing the first media in `MEDIA_LIST`, and the first localization

In [ ]:
media_w_localizations[0][4][0]

### Downloading Images and Annotations

In this cell, we're using the list of media with localizations to download JUST the frames with localizations, and of course, the localizations themselves. If you set `DOWNLOAD` to False, then the images won't actually be downloaded, but the annotations will be. This may be useful, as you may not actually need to images themselves.

In [ ]:
# np array 
media_w_localizations = np.array(media_w_localizations, dtype=object)

# Here we're going through the media with localizations, creating
# a media folder within ROOT, and then downloading just the frames
# that have localizations (ignoring those that don't, for now)

for index in range(len(media_w_localizations)):
    
    # Variables for easy access
    media = media_w_localizations[index]
    
    media_name = media[0]
    media_id = media[1]
    media_width = media[2]
    media_height = media[3]
    localizations = media[4]
    loc_id_to_class = media[5]
    
    print("Downloading frames and annotations for: ", media_name)
    
    # Download all the frames that have localizations. Images will be downloaded
    # on multiple threads to save time.
    frames_w_localizations = list(set([l.frame for l in localizations]))
    
    # Downloading each image frame with localizations, using multithreading
    # If DOWNLOAD is False, then this will just pass back the path to where the 
    # image would have been saved, which is later stored in the XML file.
    with ThreadPoolExecutor(max_workers=12) as executor:
        img_paths = [executor.submit(download_image, frame, media_name, IMAGE_DIR, api, media_id, DOWNLOAD) for 
                     frame in frames_w_localizations] 
    
    # Contains a list of frame paths on local machine
    img_paths = [future.result() for future in img_paths]
        
    # This will hold a modified version of the localizations
    # which will later be converted to a json and saved.
    locs = []
    
    # Loop through the localizations for this media
    # Creating a dict that will be used to create the XML file
    for localization in localizations:
        
        # Variables for easy access
        frame_id = localization.frame
        frame_path = IMAGE_DIR + media_name.split(".")[0] + "_" + str(frame_id) + ".jpg"
        
        # Converting the bounding boxes locations from
        # normalized to relative, and in xyxy format
        xmin = int(localization.x * media_width)
        ymin = int(localization.y * media_height)
        xmax = int(localization.width * media_width) + xmin
        ymax = int(localization.height * media_height) + ymin

        bbox = {
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax,
        }
        
        # Converting to a dict, adding variables
        loc = localization.to_dict()
        loc['bbox'] = bbox
        loc['path'] = frame_path
        
        # Not sure what's happening here. In TATOR, these localizations
        # exist and have scientific names, but it's pulling up empty
        # for a very small amount. Check this again in future...
        try:
            loc['class_category'] = loc_id_to_class[localization.id] 
        except:
            loc['class_category'] = "Unknown" 

        locs.append(loc)
        
    # Here we're creating a dict that contains all frames
    # as the key, and for each key (the frame), the localizations
    # will be stored within a nested dict. For example:
    # dict['frame_1'] = {localization_1, localization_2}
    # dict['frame_2'] = {localization_1}
    # dict['frame_3'] = {localization_1, localization_2...}
    frame_localizations = {}

    for entry in locs:
        frame = entry['frame']
        if frame in frame_localizations:
            frame_localizations[frame].append(entry)
        else:
            frame_localizations[frame] = [entry]
    
    # Exporting this nested dict as VOC pascal .xml file with the same
    # filename as the corresponding image.
    for frame in tqdm(frames_w_localizations):
        
        # Grabbing localizations for just this frame
        locs = frame_localizations[frame]
        
        # Location of file on local machine
        img_path = locs[0]['path']
        xml_path = LABEL_DIR + os.path.basename(img_path).replace(".jpg", ".xml")
        
        # Create a writer object for just this frame
        writer = Writer(img_path, width=media_width, height=media_height)

        # loop through all localizations for this frame, add with writer
        for loc in locs:
        
            bbox = loc['bbox']

            writer.addObject(loc['class_category'],
                             xmin=bbox['xmin'],
                             ymin=bbox['ymin'],
                             xmax=bbox['xmax'],
                             ymax=bbox['ymax'])
            
        writer.save(xml_path)